## BigQuery -  Github
This basic Python kernel shows you how to query the `commits` table in the GitHub Repos BigQuery dataset. We will use this information to obatin a representative sample of all the public repositories at Github. To run this notebook you will need to create a Google Cloud account and enable billing. The query takes arround three hours to complete. The result is the attached file, commit_history_raw.csv located at the data directory of the forecast folders. You can use the following links to set up your account, or you can run a sample trough my collaborative notebook published at kaggle:  
https://www.kaggle.com/code/coronate/github-monitor-data-extraction



In [ ]:
from google.cloud import bigquery
import pandas as pd
import math
import random

In [ ]:

client = bigquery.Client()
QUERY = """
        SELECT *
        FROM `bigquery-public-data.github_repos.commits`
        LIMIT 2000
        """

query_job = client.query(QUERY)

iterator = query_job.result(timeout=30)
rows = list(iterator)

commit_messages = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))

# Look at the first 10 headlines
commit_messages.head(10)

In [ ]:
query = """
    SELECT
        repo_name, 
        COUNT(*) AS commit_count,
        year, week_number
    FROM

    (SELECT
        ARRAY_TO_STRING(repo_name, ',') AS repo_name,
        FORMAT_TIMESTAMP('%Y%m%d', TIMESTAMP_SECONDS(committer.date.seconds)) AS date,
        EXTRACT(YEAR FROM TIMESTAMP_SECONDS(committer.date.seconds)) AS year,
        EXTRACT(ISOWEEK FROM TIMESTAMP_SECONDS(committer.date.seconds)) AS week_number,
        EXTRACT(MONTH FROM TIMESTAMP_SECONDS(committer.date.seconds)) AS month,
        EXTRACT(DAY FROM TIMESTAMP_SECONDS(committer.date.seconds)) AS day,
    FROM
        `bigquery-public-data.github_repos.commits`) A
        
    GROUP BY
        A.repo_name, A.year, A.week_number
"""




query_job = client.query(query)
iterator = query_job.result()
rows = list(iterator)
result_df = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))





In [ ]:
result_df["repo_name_single"] = result_df.repo_name.apply(lambda x:  x.split(",")[0].split("/")[1])
result_df["repo_author_single"] = result_df.repo_name.apply(lambda x:  x.split(",")[0])
print(len(result_df["repo_name_single"].unique())) 
print(len(result_df["repo_author_single"].unique()))

In [ ]:

df_checkpoint = result_df[["repo_author_single", "year", "week_number", "commit_count"]]
df_checkpoint.to_csv("commit_history_raw.csv")